In [19]:
from delta.tables import *
from pyspark.sql.functions import *

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 21, Finished, Available)

In [20]:
lakehousePath = "abfss://85bfc254-9abf-46cc-b1fe-943ec35b3460@msit-onelake.dfs.fabric.microsoft.com/c02dea28-20ca-432b-b6e8-39d0be76f540"
tableName = "Invoices"
tableKey = "InvoiceID"
tableKey2 = None
dateColumn = "LastEditedWhen"

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 22, Finished, Available)

In [21]:
# deltaTablePath = f"{lakehousePath}/Tables/{tableName}" 
deltaTablePath = f"{lakehousePath}/Tables/{tableName}" 
# print(deltaTablePath)

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 23, Finished, Available)

In [22]:
parquetFilePath = f"{lakehousePath}/Files/incremental/{tableName}/{tableName}.parquet"
# print(parquetFilePath)

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 24, Finished, Available)

In [23]:
df2 = spark.read.parquet(parquetFilePath)
# display(df2)

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 25, Finished, Available)

In [24]:
if tableKey2 is None:
    mergeKeyExpr = f"t.{tableKey} = s.{tableKey}"
else:
    mergeKeyExpr = f"t.{tableKey} = s.{tableKey} AND t.{tableKey2} = s.{tableKey2}"    

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 26, Finished, Available)

Check if table already exists; if it does, do an upsert and return how many rows were inserted and update; if it does not exist, return how many rows were inserted

In [25]:
if DeltaTable.isDeltaTable(spark,deltaTablePath):
    deltaTable = DeltaTable.forPath(spark,deltaTablePath)
    deltaTable.alias("t").merge(
        df2.alias("s"),
        mergeKeyExpr
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    history = deltaTable.history(1).select("operationMetrics")
    operationMetrics = history.collect()[0]["operationMetrics"]
    numInserted = operationMetrics["numTargetRowsInserted"]
    numUpdated = operationMetrics["numTargetRowsUpdated"]
else:
    df2.write.format("delta").save(deltaTablePath)  
    deltaTable = DeltaTable.forPath(spark,deltaTablePath)
    operationMetrics = history.collect()[0]["operationMetrics"]
    numInserted = operationMetrics["numOutputRows"]
    numUpdated = 0

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 27, Finished, Available)

Get the latest date loaded into the table - this will be used for watermarking; return the max date, the number of rows inserted and number updated

In [26]:
deltaTablePath = f"{lakehousePath}/Tables/{tableName}"
df3 = spark.read.format("delta").load(deltaTablePath)
maxdate = df3.agg(max(dateColumn)).collect()[0][0]
# print(maxdate)
maxdate_str = maxdate.strftime("%Y-%m-%d %H:%M:%S")

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 28, Finished, Available)

In [27]:
result = "maxdate="+maxdate_str +  "|numInserted="+str(numInserted)+  "|numUpdated="+str(numUpdated)
# result = {"maxdate": maxdate_str, "numInserted": numInserted, "numUpdated": numUpdated}
mssparkutils.notebook.exit(str(result))

StatementMeta(, bd6bdaa1-cdc5-4296-a9d7-06842037e515, 29, Finished, Available)

ExitValue: maxdate=2022-01-07 07:00:00|numInserted=6434|numUpdated=0